In [1]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 81.8 MB/s eta 0:00:00


In [2]:
import ee
import geemap
import pandas as pd
import os
import json
import geopandas as gpd
import geemap
import time

# Initialize
ee.Authenticate()
ee.Initialize(project='unicef-ccri')

In [3]:
import rasterio
import numpy as np

# Path to the GeoTIFF file
tif_path = "/content/drive/MyDrive/CCRI/ccri_repo/data/CCRI_results_misc/MHI_climate.tif"

# Calculate percentiles dynamically
with rasterio.open(tif_path) as src:
    data = src.read(1)
    data = data[data != src.nodata]  # remove nodata
    percentiles = [75, 80, 85, 90, 95]
    percentile_values = {f'_Percentile{p}': float(np.nanpercentile(data, p)) for p in percentiles}

print(percentile_values)


{'_Percentile75': 5.720572471618652, '_Percentile80': 6.068899154663086, '_Percentile85': 6.424567699432373, '_Percentile90': 6.816633701324463, '_Percentile95': 7.340970039367676}


In [4]:

# Load assets
risk_index = ee.Image("projects/unicef-ccri/assets/MHI_climate")
worldpop = ee.Image("projects/unicef-ccri/assets/worldpop_2024")
childpop = ee.ImageCollection("projects/unicef-ccri/assets/childpop_constrained_final_2024")
pop_target_res = childpop.first().projection().nominalScale()

# Mosaic child population
mosaic_pop = childpop.mosaic()


# Admin level and AOI feature collection
adm_level = 'adm0'
aois = ee.FeatureCollection(f"projects/unicef-ccri/assets/georepo_{adm_level}")

# Function to summarize population
def summarize_population(adm_level, th_name, th_value):
    exposed_pop = mosaic_pop.updateMask(risk_index.gt(th_value))
    aois = ee.FeatureCollection(f"projects/unicef-ccri/assets/georepo_{adm_level}")

    population_by_aoi = exposed_pop.reduceRegions(
        collection=aois,
        reducer=ee.Reducer.sum(),
        scale=pop_target_res.getInfo(),  # convert ee.Number to native for .reduceRegions
        crs='EPSG:4326'
    )

    def set_exposure(feature):
        return feature.set('child_population_exposed', feature.get('sum'))

    final_collection = population_by_aoi.map(set_exposure)

    task = ee.batch.Export.table.toDrive(
        collection=final_collection,
        description=f'Multi_Hazard_intensity_climate_Exposure_{adm_level}_TH{th_name}',
        fileFormat='CSV',
        selectors=['ISO3', 'child_population_exposed', 'name'],
        folder='CCRI_results_misc'
    )
    task.start()

# Use computed thresholds from step 1
thresholds = percentile_values

#loop through threosholds
for th_name, th_value in thresholds.items():
    summarize_population(adm_level, th_name, th_value)


In [5]:
df_exposed_75 = pd.read_csv('/content/drive/MyDrive/CCRI/ccri_repo/data/CCRI_results_misc/Multi_Hazard_intensity_climate_Exposure_adm0_TH_Percentile75.csv')
df_exposed_80 = pd.read_csv('/content/drive/MyDrive/CCRI/ccri_repo/data/CCRI_results_misc/Multi_Hazard_intensity_climate_Exposure_adm0_TH_Percentile80.csv')
df_exposed_85 = pd.read_csv('/content/drive/MyDrive/CCRI/ccri_repo/data/CCRI_results_misc/Multi_Hazard_intensity_climate_Exposure_adm0_TH_Percentile85.csv')
df_exposed_90 = pd.read_csv('/content/drive/MyDrive/CCRI/ccri_repo/data/CCRI_results_misc/Multi_Hazard_intensity_climate_Exposure_adm0_TH_Percentile90.csv')
df_exposed_95 = pd.read_csv('/content/drive/MyDrive/CCRI/ccri_repo/data/CCRI_results_misc/Multi_Hazard_intensity_climate_Exposure_adm0_TH_Percentile95.csv')

In [6]:
df_uregion = pd.read_csv('/content/drive/MyDrive/CCRI/ccri_repo/data/misc/UNICEF_PROG_REG_GLOBAL.csv')
df_uregion

,Regional_Grouping,Region,Region_Code,Country,ISO3Code
0,UNICEF_PROG_REG_GLOBAL,East Asia and Pacific,UNICEF_EAPRO,Cambodia,KHM
1,UNICEF_PROG_REG_GLOBAL,East Asia and Pacific,UNICEF_EAPRO,China,CHN
2,UNICEF_PROG_REG_GLOBAL,East Asia and Pacific,UNICEF_EAPRO,Cook Islands,COK
3,UNICEF_PROG_REG_GLOBAL,East Asia and Pacific,UNICEF_EAPRO,Democratic People's Republic of Korea,PRK
4,UNICEF_PROG_REG_GLOBAL,East Asia and Pacific,UNICEF_EAPRO,Fiji,FJI
...,...,...,...,...,...
153,UNICEF_PROG_REG_GLOBAL,West and Central Africa,UNICEF_WCARO,Nigeria,NGA
154,UNICEF_PROG_REG_GLOBAL,West and Central Africa,UNICEF_WCARO,Sao Tome and Principe,STP
155,UNICEF_PROG_REG_GLOBAL,West and Central Africa,UNICEF_WCARO,Senegal,SEN
156,UNICEF_PROG_REG_GLOBAL,West and Central Africa,UNICEF_WCARO,Sierra Leone,SLE


In [7]:
df_iso3_sum_75 = df_exposed_75.groupby('ISO3')['child_population_exposed'].sum().reset_index()
df_iso3_sum_80 = df_exposed_80.groupby('ISO3')['child_population_exposed'].sum().reset_index()
df_iso3_sum_85 = df_exposed_85.groupby('ISO3')['child_population_exposed'].sum().reset_index()
df_iso3_sum_90 = df_exposed_90.groupby('ISO3')['child_population_exposed'].sum().reset_index()
df_iso3_sum_95 = df_exposed_95.groupby('ISO3')['child_population_exposed'].sum().reset_index()


In [8]:
print(df_iso3_sum_75['child_population_exposed'].sum())
print(df_iso3_sum_80['child_population_exposed'].sum())
print(df_iso3_sum_85['child_population_exposed'].sum())
print(df_iso3_sum_90['child_population_exposed'].sum())
print(df_iso3_sum_95['child_population_exposed'].sum())

1585093363.9578917
1409905322.6064315
1159171503.8822572
857398012.659041
476006813.47555965


In [9]:
# Merge the three DataFrames on 'ISO3'
df_merged = df_iso3_sum_75.merge(
    df_iso3_sum_80, on='ISO3', suffixes=('_75', '_80')
).merge(
    df_iso3_sum_90, on='ISO3'
).rename(columns={'child_population_exposed': 'child_population_exposed_90'})

# Merge with df_uregion to bring in 'Region'
df_final = df_merged.merge(
    df_uregion[['ISO3Code', 'Region']],
    left_on='ISO3',
    right_on='ISO3Code',
    how='left'
).drop(columns=['ISO3Code'])

# Print the final sums for verification
print(df_iso3_sum_75['child_population_exposed'].sum())
print(df_iso3_sum_80['child_population_exposed'].sum())
print(df_iso3_sum_90['child_population_exposed'].sum())


1585093363.9578917
1409905322.6064315
857398012.659041


In [10]:
df_final.to_csv('/content/drive/MyDrive/CCRI/ccri_repo/data/misc/MHI_stat_climate.csv', index=False)

In [11]:
# Group by Region and sum exposures
df_summary_by_region = df_final.groupby('Region')[[
    'child_population_exposed_75',
    'child_population_exposed_80',
    'child_population_exposed_90'
]].sum().reset_index()

# Display the summary
print(df_summary_by_region)


                            Region  child_population_exposed_75  \
0            East Asia and Pacific                 3.082762e+08   
1      Eastern and Southern Africa                 8.806459e+07   
2          Europe and Central Asia                 4.967848e+07   
3  Latin America and the Caribbean                 8.119819e+07   
4     Middle East and North Africa                 1.547382e+08   
5                       South Asia                 5.695868e+08   
6          West and Central Africa                 2.269421e+08   

   child_population_exposed_80  child_population_exposed_90  
0                 2.750612e+08                 1.844754e+08  
1                 6.699781e+07                 3.283270e+07  
2                 4.135636e+07                 2.222151e+07  
3                 6.433401e+07                 3.754743e+07  
4                 1.451544e+08                 1.099541e+08  
5                 5.312228e+08                 3.238792e+08  
6                 2.052533e+0

In [12]:
for col in ['child_population_exposed_75', 'child_population_exposed_80', 'child_population_exposed_90']:
    df_summary_by_region[col] = (df_summary_by_region[col] / 1e6).round(2)

In [13]:
# Compute totals for all numeric columns
totals = df_summary_by_region.drop(columns='Region').sum()

# Create a total row
df_total_row = pd.DataFrame([{'Region': 'Total', **totals.to_dict()}])

# Append to the summary DataFrame
df_summary_by_region = pd.concat([df_summary_by_region, df_total_row], ignore_index=True)


In [14]:
df_summary_by_region

,Region,child_population_exposed_75,child_population_exposed_80,child_population_exposed_90
0,East Asia and Pacific,308.28,275.06,184.48
1,Eastern and Southern Africa,88.06,67.00,32.83
2,Europe and Central Asia,49.68,41.36,22.22
3,Latin America and the Caribbean,81.20,64.33,37.55
4,Middle East and North Africa,154.74,145.15,109.95
5,South Asia,569.59,531.22,323.88
6,West and Central Africa,226.94,205.25,110.42
7,Total,1478.49,1329.37,821.33


In [15]:
df_final

,ISO3,child_population_exposed_75,child_population_exposed_80,child_population_exposed_90,Region
0,ABW,0.000000e+00,0.000000e+00,0.000000e+00,NaN
1,AFG,1.388448e+07,1.225378e+07,7.746568e+06,South Asia
2,AGO,6.424742e+06,5.205267e+06,1.845728e+06,Eastern and Southern Africa
3,AIA,0.000000e+00,0.000000e+00,0.000000e+00,Latin America and the Caribbean
4,ALA,0.000000e+00,0.000000e+00,0.000000e+00,NaN
...,...,...,...,...,...
257,xSI,0.000000e+00,0.000000e+00,0.000000e+00,NaN
258,xSK,0.000000e+00,0.000000e+00,0.000000e+00,NaN
259,xSR,0.000000e+00,0.000000e+00,0.000000e+00,NaN
260,xUK,1.510291e+03,1.510119e+03,0.000000e+00,NaN
